In [ ]:
import onnx
import onnxruntime as rt
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import time

# Load the ONNX model
model = onnx.load("models/FER_Model_Adam.onnx")

# Check that the IR is well formed
onnx.checker.check_model(model)

In [ ]:
# Print a human readable representation of the graph
onnx.helper.printable_graph(model.graph)

In [ ]:
# Load the ONNX model
sess = rt.InferenceSession("models/FER_Model_Adam.onnx")
sess

In [ ]:
# Get input and output names
input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name

print("Input name: ",input_name)
print("Output name: ",output_name)

In [ ]:
classes = ['fear', 'angry', 'sad', 'neutral', 'surprise', 'disgust', 'happy']

In [ ]:
last_emotion_prob = None

def update_emotion_bars(sleep_time, emotion_probs, frame, x, y, w, last_update_time):
    global last_emotion_prob
    current_time = time.time()
    if current_time - last_update_time > sleep_time:
        # Update the emotion_probs with the latest values
        emotion_probs = emotion_probs
        last_emotion_prob = emotion_probs
        last_update_time = current_time
    else:
        # Keep the emotion_probs as it is
        if(last_emotion_prob is not None):
            emotion_probs = last_emotion_prob

    # Set up graph parameters
    bar_width = 20
    bar_spacing = 10
    max_bar_height = 100
    x_offset = x + w + 10
    y_offset = y

    # Draw the probability bars
    for i, prob in enumerate(emotion_probs):
        bar_height = int(prob * max_bar_height)
        bar_color = (0, 255, 0) if i == np.argmax(emotion_probs) else (0, 0, 255)
        cv2.rectangle(frame, (x_offset, y_offset + i * (bar_width + bar_spacing)), (x_offset + bar_height, y_offset + (i + 1) * bar_width + i * bar_spacing), bar_color, -1)
        cv2.putText(frame, f"{classes[i]}: {prob:.2f}", (x_offset + bar_height + 5, y_offset + (i + 1) * bar_width + i * bar_spacing - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    return frame, last_update_time


In [ ]:
import cv2
import os
import numpy as np
import time

last_update_time = time.time()

# Face detection setup
cascPathface = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
faceCascade = cv2.CascadeClassifier(cascPathface)
video_capture = cv2.VideoCapture(0)

while True:
    ret, frame = video_capture.read()
    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(128, 128), flags=cv2.CASCADE_SCALE_IMAGE)
    
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 255, 0), 2)
        
        face_gray = frame[y:y+h, x:x+w]
        face_gray_resized = cv2.resize(face_gray, (128, 128))

        # Convert the single channel grayscale image to a 3-channel image
        #face_gray_3channel = cv2.cvtColor(face_gray_resized, cv2.COLOR_GRAY2BGR)

        # Prepare the input for the model
        
        face_input = np.transpose(face_gray_resized, (2, 0, 1))
        face_input = np.expand_dims(face_input, axis=0)
        print(face_input.shape)
        face_input = face_input.astype(np.float32)
        face_input = face_input / 255.0


        start = time.time()
        pred_onx = sess.run([output_name], {input_name: face_input})
        end = time.time()
        print("Time taken by onnx model: ", end - start)

        print("Emotion: ", classes[np.argmax(pred_onx)])
        cv2.putText(frame, classes[np.argmax(pred_onx)], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

        # Get the probabilities for each emotion class
        emotion_probs = pred_onx[0][0]

        # Call the update_emotion_bars function with the sleep time and emotion_probs
        frame, last_update_time = update_emotion_bars(0.5, emotion_probs, frame, x, y, w, last_update_time)


    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


video_capture.release()
cv2.destroyAllWindows()